### This code illustrates the working of the VAR model to forecast soybean price based on the US Conversion prices 

In [49]:
# Loading the required packages
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
import numpy as np
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [3]:
# Read the US Conversion file
df = pd.read_csv("rate_conversion.csv")

In [4]:
df.head()

,Date,Close,Price_cny,Price_eur,Price_inr,Price_brl,Price_mxn
0,2017-11-14,978.5,6.6363,0.8476,65.410,3.3131,19.1675
1,2017-11-15,985.0,6.6235,0.8481,65.270,3.3133,19.2510
2,2017-11-16,991.0,6.6307,0.8496,65.295,3.2740,19.0590
3,2017-11-17,1004.5,6.6268,0.8478,65.015,3.2574,18.9140
4,2017-11-20,1006.5,6.6350,0.8524,65.100,3.2574,18.9937


In [5]:
df.describe()

,Close,Price_cny,Price_eur,Price_inr,Price_brl,Price_mxn
count,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000
mean,960.044248,6.697219,0.862563,68.738018,3.705080,19.211799
std,36.184460,0.232216,0.028151,2.821988,0.273822,0.553724
min,865.000000,6.269000,0.799400,63.340000,3.146500,18.012000
25%,934.437500,6.493400,0.847600,66.685000,3.501650,18.841950
50%,959.875000,6.744800,0.869850,69.167500,3.768400,19.105000
75%,992.250000,6.880200,0.884550,70.852500,3.882650,19.542900
max,1021.750000,7.165300,0.909900,74.340000,4.207700,20.890500


In [6]:
df.dtypes

Date          object
Close        float64
Price_cny    float64
Price_eur    float64
Price_inr    float64
Price_brl    float64
Price_mxn    float64
dtype: object

In [7]:
# change the datatype of the date column
df['Date'] = pd.to_datetime(df.Date , format = '%Y-%m-%d')

In [14]:
# make date column the index
data = df.drop(['Date'], axis=1)
data.index = df.Date

In [10]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [17]:
# The Johansen Test can be used to check for cointegration between a maximum of 12-time series.
coint_johansen(data,-1,1).eig

array([0.06485838, 0.03564439, 0.02673119, 0.02463259, 0.00934524,
       0.00468068])

In [18]:
# Splitting the first 80% of the data into train and the rest into test
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

In [19]:
#Loading the VAR Model
from statsmodels.tsa.vector_ar.var_model import VAR

In [20]:
# Fit the model
model = VAR(endog=train)
model_fit = model.fit()

C:\Users\vnike\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [ ]:
# Forecast using the fitted model for the next valid samples available
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [22]:
prediction

array([[9.23055952e+02, 6.73436155e+00, 8.90240612e-01, 6.97979065e+01,
        3.92904028e+00, 1.89012929e+01],
       [9.23099935e+02, 6.74246585e+00, 8.89885555e-01, 6.98289352e+01,
        3.93643644e+00, 1.88974011e+01],
       [9.23135035e+02, 6.75023671e+00, 8.89536347e-01, 6.98628376e+01,
        3.94332154e+00, 1.88931191e+01],
       [9.23163979e+02, 6.75769604e+00, 8.89194345e-01, 6.98993770e+01,
        3.94972661e+00, 1.88885345e+01],
       [9.23189171e+02, 6.76486395e+00, 8.88860763e-01, 6.99383278e+01,
        3.95568085e+00, 1.88837280e+01],
       [9.23212716e+02, 6.77175893e+00, 8.88536672e-01, 6.99794750e+01,
        3.96121167e+00, 1.88787739e+01],
       [9.23236449e+02, 6.77839798e+00, 8.88223014e-01, 7.00226141e+01,
        3.96634488e+00, 1.88737403e+01],
       [9.23261956e+02, 6.78479672e+00, 8.87920612e-01, 7.00675501e+01,
        3.97110473e+00, 1.88686897e+01],
       [9.23290600e+02, 6.79096949e+00, 8.87630175e-01, 7.01140975e+01,
        3.97551405e+00, 

In [28]:
cols = data.columns

In [56]:
# Converting thr numpy array into a dataframe 
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,6):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

In [57]:
pred.head()

,Close,Price_cny,Price_eur,Price_inr,Price_brl,Price_mxn
0,923.056,6.73436,0.890241,69.7979,3.92904,18.9013
1,923.1,6.74247,0.889886,69.8289,3.93644,18.8974
2,923.135,6.75024,0.889536,69.8628,3.94332,18.8931
3,923.164,6.7577,0.889194,69.8994,3.94973,18.8885
4,923.189,6.76486,0.888861,69.9383,3.95568,18.8837


In [53]:
pred.dtypes

Close        object
Price_cny    object
Price_eur    object
Price_inr    object
Price_brl    object
Price_mxn    object
dtype: object

In [40]:
valid.dtypes

Close        float64
Price_cny    float64
Price_eur    float64
Price_inr    float64
Price_brl    float64
Price_mxn    float64
dtype: object

In [60]:
pred.tail()

,Close,Price_cny,Price_eur,Price_inr,Price_brl,Price_mxn
86,936.154,6.974,0.89081,72.5293,3.99608,19.2153
87,936.279,6.97432,0.890881,72.5287,3.99549,19.2214
88,936.402,6.97462,0.890949,72.5275,3.99489,19.2272
89,936.522,6.97489,0.891013,72.5259,3.99429,19.233
90,936.64,6.97513,0.891074,72.5239,3.99369,19.2386


In [61]:
# Forecasting for the next one time period 
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[905.22386269   7.15679563   0.90940293  71.46518599   4.1396434
   20.02973205]]


C:\Users\vnike\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\vnike\Anaconda3\lib\site-packages\statsmodels\base\wrapper.py:35: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)
